In [1]:
FOLDER = 'C:\\Users\\Prakrith\\Desktop\\example'

# 1. Project Modules

In [2]:
import os, re, sys

In [3]:
MK_PATH = os.path.split(os.getcwd())[0] + os.sep + 'mk'
sys.path.append(MK_PATH)

In [4]:
from utils import tools
from utils.folder import Folder, Stack, Ilastik
from utils.il import RunIL
from utils.cp import RunCP
from utils.results import ResultsImage, ResultsObject, heatmap

# 2. User Variables

In [5]:
RE_TIF = re.compile("(.*)\.tif")
RE_PNG = re.compile("(.*)\.png")

In [6]:
PRBSTG_DICT = {0:'_background_prbstg2_0.png', 1:'_cell_boundary_prbstg2_1.png', 2:'_mk_prbstg2_2.png', 3:'_pplt_prbstg2_3.png'}

In [7]:
PIPELINE = tools.read_config('mk')
RULES = tools.read_config('rules')

# 3. Initialize

In [8]:
stack = Stack(FOLDER, RE_TIF)
stack.df

,filename,path
0,180417_ARW_A6_1.tif,C:\Users\Prakrith\Desktop\example


In [ ]:
time = stack.slices

22


# 4. Setup 

In [ ]:
single = Folder(stack.folder('single_images'), RE_TIF)
single.df

""


In [ ]:
stack.count
stack.check(single.count)
stack.done

False

In [ ]:
stack.unpack(single.path, 8)
single.df

C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when c

,filename,path
0,180417_ARW_A6_1-0000.tif,C:\Users\Prakrith\Desktop\example\single_images
1,180417_ARW_A6_1-0001.tif,C:\Users\Prakrith\Desktop\example\single_images
2,180417_ARW_A6_1-0002.tif,C:\Users\Prakrith\Desktop\example\single_images
3,180417_ARW_A6_1-0003.tif,C:\Users\Prakrith\Desktop\example\single_images
4,180417_ARW_A6_1-0004.tif,C:\Users\Prakrith\Desktop\example\single_images
5,180417_ARW_A6_1-0005.tif,C:\Users\Prakrith\Desktop\example\single_images
6,180417_ARW_A6_1-0006.tif,C:\Users\Prakrith\Desktop\example\single_images
7,180417_ARW_A6_1-0007.tif,C:\Users\Prakrith\Desktop\example\single_images
8,180417_ARW_A6_1-0008.tif,C:\Users\Prakrith\Desktop\example\single_images
9,180417_ARW_A6_1-0009.tif,C:\Users\Prakrith\Desktop\example\single_images


# 5. ilastik

## a) Setup

In [ ]:
ilastik = Ilastik(stack.folder('ilastik'), RE_PNG)
ilastik.df

""


In [ ]:
ilastik.count
ilastik.check(PRBSTG_DICT, single.df)
ilastik.incomplete

No probability maps found.


,filename,path
0,180417_ARW_A6_1-0000.tif,C:\Users\Prakrith\Desktop\example\single_images
1,180417_ARW_A6_1-0001.tif,C:\Users\Prakrith\Desktop\example\single_images
2,180417_ARW_A6_1-0002.tif,C:\Users\Prakrith\Desktop\example\single_images
3,180417_ARW_A6_1-0003.tif,C:\Users\Prakrith\Desktop\example\single_images
4,180417_ARW_A6_1-0004.tif,C:\Users\Prakrith\Desktop\example\single_images
5,180417_ARW_A6_1-0005.tif,C:\Users\Prakrith\Desktop\example\single_images
6,180417_ARW_A6_1-0006.tif,C:\Users\Prakrith\Desktop\example\single_images
7,180417_ARW_A6_1-0007.tif,C:\Users\Prakrith\Desktop\example\single_images
8,180417_ARW_A6_1-0008.tif,C:\Users\Prakrith\Desktop\example\single_images
9,180417_ARW_A6_1-0009.tif,C:\Users\Prakrith\Desktop\example\single_images


## b) Processing

In [ ]:
RunIL(ilastik.incomplete)
ilastik.df

C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Poss

# 6. Cellprofiler

## MK

In [ ]:
out = stack.folder('output')

In [ ]:
RunCP(PIPELINE, stack.path, 'mk.txt', single.list + ilastik.list, out, rules=RULES)

# 7. Results

## results_Image.csv

In [ ]:
res = ResultsImage(out, 'results_Image.csv', time)

In [ ]:
res.df

In [ ]:
x = res.analyze()

## results_mk_filter.csv

In [ ]:
res2 = ResultsObject(out, 'results_mk_filter.csv', time, 'MK')

In [ ]:
res2.df

In [ ]:
y = res2.analyze()

## results_pplt_filter.csv

In [ ]:
res3 = ResultsObject(out, 'results_pplt_filter.csv', time, 'PPLT')

In [ ]:
res3.df

In [ ]:
z = res3.analyze()

In [ ]:
heatmap(x,y,z,out)

In [ ]:
tools.move_results(out)